In [22]:
# pip install firebase_admin
#pip install datetime

In [23]:
import firebase_admin
from firebase_admin import credentials, firestore

In [24]:
if not firebase_admin._apps:
	cred = credentials.Certificate("./infinite-hope-444810-i6-firebase-adminsdk-yd3a1-b3d8b4f2ba.json")
	firebase_admin.initialize_app(cred)

db = firestore.client()

In [25]:
import json 

# get boundary coordinates from the database
def fetch_coordinates():
    docs = db.collection('areas').stream()
    coordinates_per_area = {}

    for doc in docs:
        area = doc.to_dict()
        doc_id = doc.id
        title = area.get('title')
        print(area)
        markers = area.get('markers')
        list_markers = []
        
        if isinstance(markers, str):
            markers_lat_lon = json.loads(markers)
            list_markers.append(markers_lat_lon)
            
        else : 
            if isinstance(markers, object):
            
                for m in markers : 
                    geopoint = m.get('geopoint')
                    latitude = geopoint.latitude
                    longitude =geopoint.longitude
                    
                    if latitude is not None and longitude is not None:
                        list_markers.append([latitude,longitude])

        coordinates_per_area[title] = list_markers
    return coordinates_per_area



coordinates_per_area = fetch_coordinates()
print(coordinates_per_area)
# could also be getting the data from the database 


{'description': '2334', 'title': 'a1', 'status': 0, 'markers': [{'geohash': 'u0qwmvvzw', 'geopoint': <google.cloud.firestore_v1._helpers.GeoPoint object at 0x000002843D75AC90>}, {'geohash': 'u0qwmyn61', 'geopoint': <google.cloud.firestore_v1._helpers.GeoPoint object at 0x000002843C67D450>}, {'geohash': 'u0qwmyjsd', 'geopoint': <google.cloud.firestore_v1._helpers.GeoPoint object at 0x000002843D79DBD0>}, {'geohash': 'u0qwmyj4m', 'geopoint': <google.cloud.firestore_v1._helpers.GeoPoint object at 0x000002843D7485D0>}]}
{'description': '3455', 'title': 't12', 'status': 0, 'markers': [{'geohash': 'u0qwmz6gf', 'geopoint': <google.cloud.firestore_v1._helpers.GeoPoint object at 0x000002843C67CF10>}, {'geohash': 'u0qwmz601', 'geopoint': <google.cloud.firestore_v1._helpers.GeoPoint object at 0x000002843D749390>}, {'geohash': 'u0qwmz4ts', 'geopoint': <google.cloud.firestore_v1._helpers.GeoPoint object at 0x000002843D74BC90>}, {'geohash': 'u0qwmz73b', 'geopoint': <google.cloud.firestore_v1._helpers

In [26]:
import datetime
import random 

# generating realalistic data
# Update the GPS data generation to limit the total time span to 5 minutes
def generate_limited_gps_data(boundary_coords, total_duration_minutes=5, interval_seconds=5):
    gps_data = []
    base_time = datetime.datetime.now()
    lat_min = min(coord[0] for coord in boundary_coords)
    lat_max = max(coord[0] for coord in boundary_coords)
    lon_min = min(coord[1] for coord in boundary_coords)
    lon_max = max(coord[1] for coord in boundary_coords)

    statuses = ["driving", "mowing", "calculating direction", "awaiting instructions"]

    total_seconds = total_duration_minutes * 60
    num_points = total_seconds // interval_seconds

    for i in range(num_points):
        # Generate a random coordinate within the boundary
        latitude = random.uniform(lat_min, lat_max)
        longitude = random.uniform(lon_min, lon_max)
        
        # Simulate timestamps and movements
        timestamp = base_time + datetime.timedelta(seconds=i * interval_seconds)
        time_spent = random.randint(10, interval_seconds)  # seconds spent at coordinate
        annotations = random.choice(statuses)
        
        gps_data.append({
            "Latitude": round(latitude, 8),
            "Longitude": round(longitude, 8),
            "Timestamp": timestamp.isoformat(),
            "Time Spent": time_spent,
            "Annotations": annotations
        })
    
    return gps_data

In [27]:
# write the data to firestore
def write_data_to_firestore(current_time, data, area):
    
    doc_ref = db.collection("session").document()
    doc_ref.set({
        'session_time' : current_time,
        'datapoints' : data,
        'area' : area	
    })
    

In [ ]:
# combining ressources to write (fake) data to firestore
for area, boundary_coords in coordinates_per_area.items():
    print(area)
    if boundary_coords and len(boundary_coords) > 1:  # Ensure boundary_coords is not empty
        print(len(boundary_coords), area)
        limited_gps_data = generate_limited_gps_data(boundary_coords, total_duration_minutes=4, interval_seconds=20)
        time = datetime.datetime.now()
        write_data_to_firestore(time, limited_gps_data, area)


a1
4 a1
t12
4 t12
area3
test 4
aaa
Area 22
Area 10


: 